# **Creating a Keyword Search Method**
I've gotten a prototype for "neural" search working pretty well, but I want to test out something a little more traditional: keyword search! Postgres apparently supports some full-text search capabilities, so I want to try them out within this notebook. 

# **Setup**
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [1]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

d:\data\programming\neural-needledrop\api


Now I'll import some necessary modules:

In [2]:
# General import statements
import pandas as pd
import time
from pandas_gbq import read_gbq
from sqlalchemy import create_engine, MetaData
from IPython.display import Markdown, display
from sqlalchemy.orm import sessionmaker, declarative_base
from tqdm import tqdm
from pathlib import Path
from google.cloud import storage
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy.stats import kendalltau
import plotly.express as px
from datetime import datetime

# Importing modules custom-built for this project
from utils.settings import (
    GBQ_PROJECT_ID,
    GBQ_DATASET_ID,
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB,
    LOG_TO_CONSOLE,
)
from utils.logging import get_logger
from utils.postgres import query_postgres, upload_to_table
from utils.postgres_queries import (
    most_similar_embeddings_to_text_filtered,
    create_filters,
    get_most_similar_transcriptions_filtered,
    fetch_text_for_segments
)
from utils.search import neural_search, keyword_search

# Set up a logger for this notebook
logger = get_logger("postgres_notebook", log_to_console=LOG_TO_CONSOLE)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trevb_b7z2dw1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Next up: we're going to set up the Postgres engine via SQLAlchemy!

In [3]:
# Create the connection string to the database
postgres_connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the connection engine
engine = create_engine(postgres_connection_string)
metadata = MetaData()
session = sessionmaker(bind=engine)()
Base = declarative_base()

# **Ensuring Full-Text Index Exits**
One thing that I need to do: actually make sure that the `transcriptions` table contains a `tsvector` column, and that we have an index created for this column. This enables the full-text searching that I want to do! 

In [ ]:
# Select one row from the transcriptions table and convert it to a dictionary
transcription_record_example = query_postgres("SELECT * FROM transcriptions LIMIT 1", engine).iloc[0].to_dict()

# Check to see if the ts_vec column is present in the transcriptions table
tsvector_column_exists = "ts_vec" in transcription_record_example

# Print the result
print(f"Does the ts_vec column exist in the transcriptions table? {tsvector_column_exists}")

If there isn't a `ts_vec` column within the `transcriptions` table, we'll create it. 

In [ ]:
# We're only going to run this notebook if the ts_vec column doesn't exist
if not tsvector_column_exists:
    # Define a query that'll add a tsvector column to the transcriptions table
    add_tsvector_column_query = """
    ALTER TABLE transcriptions
    ADD COLUMN ts_vec TSVECTOR
    GENERATED ALWAYS AS (
        to_tsvector('english', text) 
    ) STORED;
    """

    # Execute the query
    query_postgres(add_tsvector_column_query, engine)

Next up: we're going to check if there's an index on this column. 

In [ ]:
# Define a query to check if the index exists
check_index_query = """
SELECT 
    EXISTS (
        SELECT 1
        FROM pg_class c
        JOIN pg_namespace n ON n.oid = c.relnamespace
        WHERE c.relname = 'transcriptions_text_idx'
        AND n.nspname = 'public'
    );
"""

# Execute the query and store the result
index_exists = query_postgres(check_index_query, engine).iloc[0].exists

# Print the result
print(f"Does the transcriptions_text_idx index exist? {index_exists}")


If the index doesn't exist, then we're going to add it. 

In [ ]:
# If the index_exists variable is False, then we'll create the index
if not index_exists:

    # Set the maintenance_work_mem to 6GB
    query_postgres("SET maintenance_work_mem = '6GB';", engine)

    # Create a GIN index on the ts_vec column
    create_index_query = (
        """CREATE INDEX transcriptions_text_idx ON transcriptions USING GIN(ts_vec);"""
    )

    # Execute the query
    query_postgres(create_index_query, engine)

# **Testing Full-Text Search**
Now that I've got a `tsvector` column and a GIN index, I can test the full-text search. 

In [ ]:
# Parameterize the function
query = "jim carrey"
release_date_filter = [datetime(2010, 1, 1), datetime(2023, 1, 1)]
review_score_filter = [6, 9]
video_type_filter = ["album_review"]
n_results = 100
n_most_similar_videos = 5
n_top_segments_per_video = 3


# Get the most similar transcriptions to the query
most_similar_transcriptions_df = get_most_similar_transcriptions_filtered(
    query=query,
    engine=engine,
    release_date_filter=release_date_filter,
    review_score_filter=review_score_filter,
    video_type_filter=video_type_filter,
    n_results=n_results,
    include_text=False
)

most_similar_transcriptions_df.head(3)

Now that I've got some of the most similar text results, I can aggregate some information about them. 

In [ ]:
# Aggregate the transcription stats
aggregated_transcription_stats_df = most_similar_transcriptions_df.groupby("url").agg(
    median_rank=("rank", "median"),
    mean_rank=("rank", "mean"),
    n_results=("rank", "count"),
)

# Determine the z-score of the count of results
aggregated_transcription_stats_df["count_z_score"] = (
    aggregated_transcription_stats_df["n_results"]
    - aggregated_transcription_stats_df["n_results"].mean()
) / aggregated_transcription_stats_df["n_results"].std()

# Now, make a "weighted median", which is the median rank weighted by the z-score of the count of results
aggregated_transcription_stats_df["weighted_median_rank"] = (
    aggregated_transcription_stats_df["median_rank"]
    * aggregated_transcription_stats_df["count_z_score"]
)

# Sort the dataframe by the weighted median rank
aggregated_transcription_stats_df = (
    aggregated_transcription_stats_df.sort_values(
        "weighted_median_rank", ascending=False
    )
    .head(n_most_similar_videos)
    .reset_index()
    .copy()
)

Finally, I'm going to retrieve some video metadata and text. I'll start with the metadata.

In [ ]:
# Uplaod the dataframe to the database
aggregated_transcription_stats_df.to_sql("most_similar_transcriptions_temp", engine, if_exists="replace")

# Create a query to get the video metadata
video_metadata_query = """
SELECT
    video.*
FROM
    video_metadata video
JOIN
    most_similar_transcriptions_temp transcriptions
ON
    video.url = transcriptions.url
"""

# Execute the query
most_similar_videos_metadata_df = query_postgres(video_metadata_query, engine)

# Re-join the aggreageted transcription stats with the video metadata
aggregated_transcription_stats_with_metadata_df = aggregated_transcription_stats_df.merge(
    most_similar_videos_metadata_df, on="url"
)

# Show the results
aggregated_transcription_stats_with_metadata_df.head(3)

Next, the text: 

In [ ]:
transcriptions_to_fetch_df = most_similar_transcriptions_df.merge(
    aggregated_transcription_stats_with_metadata_df[["url"]],
    on="url",
)

# Upload a temporary table to the database
transcriptions_to_fetch_df.to_sql(
    "transcriptions_to_fetch_temp", engine, if_exists="replace"
)

# Now, we'll fetch the text for the segments
fetch_text_for_segments_query = """
SELECT
    transcriptions.text,
    transcriptions.url,
    transcriptions.segment_id AS id
FROM
    transcriptions_to_fetch_temp segments
LEFT JOIN
    transcriptions
ON
    transcriptions.url = segments.url
    AND transcriptions.segment_id = segments.id
"""

# Execute the query
transcriptions_with_text_df = query_postgres(fetch_text_for_segments_query, engine)

# Aggregate this into the transcriptions_to_fetch_df dataframe
transcriptions_to_fetch_df = transcriptions_to_fetch_df.merge(
    transcriptions_with_text_df, on=["url", "id"]
)

# Aggregate the text into a list
transcriptions_to_fetch_df = (
    transcriptions_to_fetch_df.sort_values("rank", ascending=False)
    .groupby("url")
    .agg(top_segment_chunks=("text", lambda x: list(x)[:n_top_segments_per_video]))
    .reset_index()
)

# Add this to the aggregated_transcription_stats_with_metadata_df dataframe
aggregated_transcription_stats_with_metadata_and_text_df = aggregated_transcription_stats_with_metadata_df.merge(
    transcriptions_to_fetch_df, on="url"
)

Finally, we can show off the results:

In [ ]:
for index, row in aggregated_transcription_stats_with_metadata_and_text_df.head(3).iterrows():
    display(Markdown(f"**{row['title']}**"))
    display(Markdown("\n".join([f"* {chunk}" for chunk in row['top_segment_chunks']])))


# Functionalized Version
Now, I've gone ahead and turned this into a method of its own: 

In [12]:
# Parameterize the function
query = "beautiful piano strings, slipping through the trees like dreams"
release_date_filter = [datetime(2010, 1, 1), datetime(2023, 1, 1)]
review_score_filter = [6, 9]
video_type_filter = ["album_review"]
n_results = 100
n_most_similar_videos = 5
n_top_segments_per_video = 3

release_date_filter = None
review_score_filter = None
video_type_filter = None

# Run the query
keyword_search_results_json = keyword_search(
    query=query,
    release_date_filter=release_date_filter,
    review_score_filter=review_score_filter,
    video_type_filter=video_type_filter,
    n_results_to_consider=n_results,
    n_most_similar_videos=n_most_similar_videos,
    n_top_segments_per_video=n_top_segments_per_video,
)

# Convert to a dataframe
keyword_search_results_df = pd.DataFrame(json.loads(keyword_search_results_json))



The most_similar_transcription_segments_query is:


    SELECT
    video_metadata.title,
    transcriptions.url,
    
    transcriptions.segment_id AS id,
    transcriptions.segment_seek,
    transcriptions.segment_start,
    transcriptions.segment_end,
    ts_rank_cd(ts_vec, to_tsquery('english', 'beautiful | piano | strings, | slipping | trees | dreams | beautiful <-> piano | piano <-> strings, | strings, <-> slipping | slipping <-> trees | trees <-> dreams | beautiful <-> piano <-> strings, | piano <-> strings, <-> slipping | strings, <-> slipping <-> trees | slipping <-> trees <-> dreams')) AS rank
    FROM
    transcriptions
    LEFT JOIN
    video_metadata
    ON
    video_metadata.url = transcriptions.url
    WHERE
    ts_vec @@ to_tsquery('english', 'beautiful | piano | strings, | slipping | trees | dreams | beautiful <-> piano | piano <-> strings, | strings, <-> slipping | slipping <-> trees | trees <-> dreams | beautiful <-> piano <-> strings, | piano <-> strings, <-> slippin

In [14]:
# Show the results
keyword_search_results_df.iloc[0].top_segment_chunks

[' This is a no frills, no bells and whistles just clean cut straight forward set of folkie, low key tunes with beautiful poetic songwriting killer singing the instrumentation again bear almost skeletal at points but still compelling still moving still emotionally potent this record is beautiful from corner to corner beautiful beautiful beautiful beautiful beautiful beautiful beautiful.']